# Welcome

Using this notebook, you can see how different keyboard layouts will work for you. 

In [36]:
import os
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

## File utils

In [37]:
def find_files(root_folder, extension):
    res = []
    for root, dirs, files in os.walk(root_folder, topdown=False):
        for name in files:
            if name.endswith(extension):
                print os.path.join(root, name)
                res.append(os.path.join(root, name))
    return res    

In [25]:
def get_char_hist(string):
    letter_hist = Counter(string.replace('\n', ''))
    return letter_hist

In [26]:
def file_to_string(path):
    with open(path, 'r') as f:
        string = f.read().replace('\n', '')
    return string.replace(' ', '')

In [27]:
def accumulate_hists(h1, h2):
    for key in h1.keys():
        if key in k2.keys():
            h1[key] += h2[key]
    return h1

## Function which overlays the key frequency on the layout and returns the mask

In [70]:
def generate_layout_map(freqeuncy_list, original_layout_shape):
    block_size = 60
    row_blocks = [13, 13, 11, 10]
    row_offsets = [0, 90, 105, 135]
    mask = np.zeros(original_layout_shape)
    count = 0
    for row in range(4):
        for i in range(row_blocks[row]):
            mask[row*block_size:(row+1)*block_size,
                 i*block_size + row_offsets[row]:(i+1)*block_size + row_offsets[row]] = freqeuncy_list[row][i]
    return mask

## Function to map the character histogram to frequency list based on layout

In [29]:
def hist_to_freq_list(hist, layout):
    freq_list = [np.zeros((13)),
                 np.zeros((13)),
                 np.zeros((11)),
                 np.zeros((10))]
    for row_ind, row in enumerate(layout):
        row_list = []
        count = 0
        for i in range(0, len(row), 2):
            freq_list[row_ind][count] = hist[layout[row_ind][i]] + hist[layout[row_ind][i+1]]
            count += 1
    return freq_list

## layouts

In [30]:
workman = ['`~1!2@3#4$5%6^7&8*9(0)-_=+', 'qQdDrRwWbBjJfFuUpP;:[{]}\|', 'aAsShHtTgGyYnNeEoOiI\'\"', 'zZxXmMcCvVkKlL,<.>/?']
dvorak = ['`~1!2@3#4$5%6^7&8*9(0)[{]}', '\'\",<.>pPyYfFgGcCrRlL/?=+\|', 'aAoOeEuUiIdDhHtTnNsS-_', ';:qQjJkKxXbBmMwWvVzZ']
colemac = ['`~1!2@3#4$5%6^7&8*9(0)-_=+', 'qQwWfFpPgGjJlLuUyY;:[{]}\|', 'aArRsStTdDhHnNeEiIoO\'\"', 'zZxXcCvVbBkKmM,<.>/?']
qwerty = ['`~1!2@3#4$5%6^7&8*9(0)-_=+', 'qQwWeErRtTyYuUiIoOpP[{]}\|', 'aAsSdDfFgGhHjJkKlL;:\'\"', 'zZxXcCvVbBnNmM,<.>/?']

# Let the party begin :P

### Read all of the files you are interested

In [100]:
files = find_files('~/git/', '.cpp')

### Create a string from all of the files 

In [96]:
string = ''
for f in files:
    if f.split('/')[-1].startswith('.'):
        continue
    string += file_to_string(f)


### Create the character histogram

In [ ]:
h = get_char_hist(string)

### Load the keyboard layout images and threshold them

In [97]:
qwerty_image = cv2.imread('../layouts/KB_US-QWERTY_with_AltGr.png', 0)
dvorak_image = cv2.imread('../layouts/KB_US-Dvorak_with_AltGr.png', 0)
colemac_image = cv2.imread('../layouts/KB_US-Colemak_with_AltGr.png', 0)
workman_image = cv2.imread('../layouts/KB_US-Workman_with_AltGr.png', 0)
qwerty_image[np.where(qwerty_image > 100)] = 255
dvorak_image[np.where(dvorak_image > 100)] = 255
colemac_image[np.where(colemac_image > 100)] = 255
workman_image[np.where(workman_image > 100)] = 255

### Make the stroke map of the keys and overlay them on the layout images

In [99]:
smoothing_kernel = np.ones((11, 11), np.float32)/121

f_list = hist_to_freq_list(h, colemac)
mask = generate_layout_map(f_list, (300,900))
mask = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))
mask_smooth = cv2.filter2D(mask, -1, smoothing_kernel)
cv2.imwrite('colemac.png', mask_smooth * colemac_image)

f_list = hist_to_freq_list(h, dvorak)
mask = generate_layout_map(f_list, (300,900))
mask = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))
mask_smooth = cv2.filter2D(mask, -1, smoothing_kernel)
cv2.imwrite('dvorak.png', mask_smooth * dvorak_image)

f_list = hist_to_freq_list(h, workman)
mask = generate_layout_map(f_list, (300,900))
mask = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))
mask_smooth = cv2.filter2D(mask, -1, smoothing_kernel)
cv2.imwrite('workman.png', mask_smooth * workman_image)

f_list = hist_to_freq_list(h, qwerty)
mask = generate_layout_map(f_list, (300,900))
mask = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))
mask_smooth = cv2.filter2D(mask, -1, smoothing_kernel)
cv2.imwrite('qwerty.png', mask_smooth * qwerty_image)

True